# Initalize libraries

## Import libraries

In [ ]:
import sys, os
import time
from os.path import join
from os import path
from importlib import reload
from getpass import getuser
from glob import glob
from tqdm.auto import tqdm

# Data
import xarray as xr
import h5py
import numpy as np
from nexusformat.nexus import *

# Plotting
import imageio
import matplotlib.pyplot as plt
import fabio
import skimage.morphology

# scipy
from scipy.ndimage.filters import gaussian_filter
import scipy
from scipy import stats

# pyFAI
import pyFAI
from pyFAI.azimuthalIntegrator import AzimuthalIntegrator
from pyFAI.detectors import Detector

# Self-written libraries
sys.path.append(join(os.getcwd(), "library"))
import reconstruct as reco
import fthcore as fth
import support_functions as sup
import interactive
from interactive import cimshow
import reconstruct_rb as rec
import reconstruct as reco

plt.rcParams["figure.constrained_layout.use"] = True  # replaces plt.tight_layout

In [ ]:
# Is there a GPU?
try:
    # Cupy
    import cupy as cp
    import cupyx as cpx

    GPU = True

    print("GPU available")

    # Self-written library
    import CCI_core_cupy as cci
    import Phase_Retrieval as PhR
except:
    GPU = False
    import CCI_core as cci

    print("GPU unavailable")

In [ ]:
# interactive plotting
import ipywidgets

%matplotlib widget

# Auto formatting of cells
#%load_ext jupyter_black

## Experiment specific Functions

In [ ]:
PROPOSAL = 11018117
USER = getuser()

### Loading data

In [ ]:
BASEFOLDER = "/DATA/beamtimes/PETRA/2308_MaxP04/%s/" % PROPOSAL
subfolder = "XPCS"
sample_name = "XPCS_Aug2023"


# Load image files
def load_images(im_id):
    """
    Load ccd images from nxs files
    """

    fname = join(BASEFOLDER, "raw", subfolder, "%s_%05d.nxs" % (sample_name, im_id))

    with nxload(fname) as f:
        im_out = np.array(f["scan"]["data"]["ccd"][()])
        print("Loaded: %s (%d frames)" % (fname, im_out.shape[0]))
    return im_out.squeeze()


# Load any kind of data from collection
def load_collection(scan_id, field):
    fname = join(BASEFOLDER, "raw", subfolder, "%s_%05d.nxs" % (sample_name, scan_id))

    with nxload(fname) as f:
        data = np.array(f["scan"]["instrument"]["collection"][field][()])
        print("Loaded: %s" % (fname))
    return np.squeeze(data)


# Load mono energy
def load_energy(scan_id):
    fname = join(BASEFOLDER, "raw", subfolder, "%s_%05d.nxs" % (sample_name, scan_id))

    with nxload(fname) as f:
        data = np.array(f["scan"]["instrument"]["mono"]["energy"][()])
        print("Loaded: %s" % (fname))
    return np.squeeze(data)


# Load any kind of data from measurements
def load_data(scan_id, field):
    fname = join(BASEFOLDER, "raw", subfolder, "%s_%05d.nxs" % (sample_name, scan_id))

    with nxload(fname) as f:
        data = np.array(f["scan"]["data"][field][()])
        print("Loaded key %s of %s" % (field, fname))
    return np.squeeze(data)


# Full image loading procedure
def load_processing(im_id, crop=None):
    """
    Loads images, averaging of two individual images (scans in tango consist of two images),
    padding to square shape, Additional cropping (optional)
    """

    # Load data
    images = load_images(im_id)

    # Zeropad to get square shape
    images = sup.padding(images)

    # Calculate mean
    if images.ndim > 2:
        image = np.mean(images, axis=0)
    else:
        image = images.copy()

    # Optional cropping
    if crop is not None:
        images = images[:, :crop, :crop]
        image = image[:crop, :crop]

    return image, images

### Loading, saving fth & cdi data

In [ ]:
# Saving of log files for fth and cdi recos
def save_fth_h5():
    # Save h5
    data = {}
    data["im_id"] = im_id
    data["topo_id"] = topo_id
    data["topo_centered"] = topo_c
    data["im_centered"] = im_c
    data["holo"] = holo
    data["sum_b"] = sum_b
    data["recon"] = recon
    data["factor"] = factor
    data["offset"] = offset
    data["center"] = center
    data["roi"] = roi
    data["prop_dist"] = prop_dist
    data["phase"] = phase
    data["mask_bs"] = mask_bs
    data["bs_diam"] = bs_diam

    filename = join(
        folder_general, "Logs", "Data_ImId_%04d_RefId_%s_%s" % (im_id, topo_id, USER)
    )
    print("Now Saving: %s" % filename)
    cci.create_hdf5(data, filename)


def save_cdi_h5():
    # Save h5
    data = {}
    data["im_id"] = im_id
    data["topo_id"] = topo_id
    data["pos"] = pos
    data["neg"] = neg
    data["factor"] = factor
    data["offset"] = offset
    data["center"] = center
    data["roi"] = roi
    data["prop_dist"] = prop_dist_cdi
    data["phase"] = phase_cdi
    data["mask_bs"] = mask_bs
    data["bs_diam"] = bs_diam_cdi
    data["supportmask"] = supportmask
    data["mask_pixel"] = mask_pixel
#    data["p"] = p
#    data["n"] = n
    data["p_pc"] = p_pc
    data["n_pc"] = n_pc
#    data["gamma_p"] = gamma_p
#    data["gamma_n"] = gamma_n

    filename = join(
        folder_general,
        "Logs",
        "Data_ImId_%04d_RefId_%s_cdi_%s" % (im_id, topo_id, USER),
    )
    print("Now Saving: %s" % filename)
    cci.create_hdf5(data, filename)
    return


def save_topo_holo(topo_c, pos_id, neg_id):
    """
    Save only topo holos which can be later used for single helicity reconstructions
    """
    data = {}
    data["pos_id"] = pos_id
    data["neg_id"] = neg_id
    data["topo"] = topo_c

    filename = join(
        folder_general,
        "Topos",
        "Topo_ImId_%04d_RefId_%04d_cdi_%s" % (pos_id, neg_id, USER),
    )
    print("Now Saving: %s" % filename)
    cci.create_hdf5(data, filename)
    return


def load_topo_holo(pos_id, neg_id):
    """
    Load topo holos for single helicity reconstructions
    """
    fname = join(
        folder_general,
        "Topos",
        "Topo_ImId_%04d_RefId_%04d_cdi_%s.hdf5" % (pos_id, neg_id, USER),
    )

    with h5py.File(fname, "r") as f:
        im_out = f["topo"][()]
    return im_out


def load_cdi(im_id, topo_id):
    """
    Load cdi dataset
    """
    fname = join(
        folder_general,
        "Logs",
        "Data_ImId_%04d_RefId_%s_cdi_%s.hdf5" % (im_id, topo_id, USER),
    )

    with h5py.File(fname, "r") as f:
        data_cdi = {}
        data_cdi["im_id"] = f["im_id"][()]
        data_cdi["topo_id"] = f["topo_id"][()]
        data_cdi["topo_centered"] = f["topo_centered"][()]
        data_cdi["im_centered"] = f["im_centered"][()]
        data_cdi["pos"] = f["pos"][()]
        data_cdi["neg"] = f["neg"][()]
        data_cdi["recon"] = f["recon"][()]
        data_cdi["factor"] = f["factor"][()]
        data_cdi["offset"] = f["offset"][()]
        data_cdi["shift"] = f["shift"][()]
        data_cdi["center"] = f["center"][()]
        data_cdi["roi"] = f["roi"][()]
        data_cdi["prop_dist"] = f["prop_dist"][()]
        data_cdi["phase"] = f["phase"][()]
        data_cdi["mask_bs"] = f["mask_bs"][()]
        data_cdi["bs_diam"] = f["bs_diam"][()]
        data_cdi["supportmask"] = f["supportmask"][()]
        data_cdi["mask_pixel"] = f["mask_pixel"][()]
        data_cdi["p"] = f["p"][()]
        data_cdi["n"] = f["n"][()]
        data_cdi["p_pc"] = f["p_pc"][()]
        data_cdi["n_pc"] = f["n_pc"][()]
        data_cdi["gamma_p"] = f["gamma_p"][()]
        data_cdi["gamma_n"] = f["gamma_n"][()]
    return data_cdi

### Masking

In [ ]:
# Still experimental
from dipy.segment.mask import median_otsu


# Still experimental
def automated_beamstop(hologram, grad_thres, radius, expand, method="intensity"):
    # Different methods for filtering of beamstop
    if method == "intensity":
        # Some filters
        pass
    elif method == "gradient":
        hologram = np.mean(np.abs(np.gradient(hologram)), axis=0)

    # Thresholding with gradient
    hologram_mask = hologram < grad_thres

    # Draw beamstop only up to given radius
    hologram_mask = hologram_mask * cci.circle_mask(
        image.shape,
        [image.shape[0] / 2, image.shape[1] / 2],
        radius,
        sigma=None,
    )
    hologram_mask = hologram_mask.astype(bool)

    # Morphological operations to filter reference modulations as these also lead to strong intensity gradients
    # close the "dots" of the ref modulations
    footprint = skimage.morphology.disk(2)
    hologram_mask = skimage.morphology.erosion(hologram_mask, footprint)

    # Filter remainings of ref modulations
    hologram_mask = skimage.morphology.remove_small_objects(
        hologram_mask, min_size=2000
    )

    # Expand Mask
    footprint = skimage.morphology.disk(expand)
    hologram_mask = skimage.morphology.dilation(hologram_mask, footprint)

    # Fill up small holes in the mask
    hologram_mask = scipy.ndimage.binary_fill_holes(
        hologram_mask, structure=np.ones((5, 5))
    )

    return hologram_mask


def automated_beamstop_center(hologram, expand):
    # "Flatten" hologram
    hologram[hologram < 0] = 0
    hologram = hologram + 1
    hologram = np.log10(hologram)

    # Prepare raw mask using otsu threshold method
    _, hologram_mask = median_otsu(hologram, median_radius=1, numpass=1)

    # Morphological operations to filter reference modulations
    # close the "dots" of the ref modulations
    footprint = skimage.morphology.disk(2)
    hologram_mask = skimage.morphology.erosion(
        (1 - hologram_mask).astype(bool), footprint
    )

    # Filter remainings of ref modulations
    hologram_mask = skimage.morphology.remove_small_objects(
        hologram_mask.astype(bool), min_size=2000
    )

    # Expand mask to desired size
    footprint = skimage.morphology.disk(expand)
    hologram_mask = skimage.morphology.dilation(hologram_mask, footprint)

    hologram_mask = 1 - skimage.morphology.remove_small_objects(
        (1 - hologram_mask).astype(bool), min_size=2000
    )

    return hologram_mask

In [ ]:
def create_supportmask(support_coordinates, shape):
    """
    Create cdi support mask from a combination of multiple circular apertures

    Parameter
    =========
    support_coordinates: list of tuples
        Contains center coordinates and radius of each aperture [(yc_1,xc_1,r_1),(yc_2,xc_2,r_2),...]
    shape : int tuple
        shape/dimension of output array

    Output
    ======
    supportmask: array
        composed binary mask where circular apertures are "1"
    ======
    author: ck 2023

    """

    # Create support mask
    supportmask = np.zeros(shape)
    for i in range(len(support_coordinates)):
        supportmask += cci.circle_mask(
            supportmask.shape,
            [support_coordinates[i][0], support_coordinates[i][1]],
            support_coordinates[i][2],
        )

    return supportmask

In [ ]:
from matplotlib.path import Path


def create_single_polygon_mask(shape, coordinates):
    """
    Creates a polygon mask from coordinates of corner points

    Parameter
    =========
    shape : int tuple
        shape/dimension of output array
    coordinates: nested list
        coordinates of polygon corner points [(yc_1,xc_1),(yc_2,xc_2),...]


    Output
    ======
    mask: array
        binary mask where filled polygon is "1"
    ======
    author: ck 2023
    """

    x, y = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]))
    x, y = x.flatten(), y.flatten()

    points = np.vstack((x, y)).T

    path = Path(coordinates)
    mask = path.contains_points(points)
    mask = mask.reshape(shape)
    return mask


def create_polygon_mask(shape, coordinates):
    """
    Creates multiple polygon masks from set of coordinates of corner points

    Parameter
    =========
    shape : int tuple
        shape/dimension of output array
    coordinates: nested list
        coordinates of polygon corner points for multiple polygons
        [[(yc_1,xc_1),(yc_2,xc_2),...],[(yc_1,xc_1),(yc_2,xc_2),...]]

    Output
    ======
    mask: array
        binary mask where filled polygons are "1"
    ======
    author: ck 2023
    """

    if len(coordinates) == 1:
        mask = create_single_polygon_mask(shape, coordinates[0])

    # Loop over coordinates
    elif len(coordinates) > 1:
        mask = np.zeros(shape)
        for coord in coordinates:
            mask = mask + create_single_polygon_mask(shape, coord)
            mask[mask > 1] = 1

    return mask


def load_poly_masks(shape, polygon_name_list):
    """
    Loads set of polygon masks based on stored coordinates

    Parameter
    =========
    shape : tuple
        shape of output mask
    polygon_name_list : list
        keys of different mask coordinates to load

    Output
    ======
    mask: array
        binary mask where filled polygons are "1"
    ======
    author: ck 2023
    """

    mask = []

    # Load dictionary of coordinates
    mask_coordinates = load_poly_coordinates()

    # Loop over relevant mask keys
    for polygon_name in polygon_name_list:
        coord = mask_coordinates[polygon_name]
        mask.append(create_polygon_mask(shape, coord).astype(float))

    # Combine all individual mask layers
    mask = np.array(mask)
    mask = np.sum(mask, axis=0)
    mask[mask > 1] = 1

    return mask

### Other

In [ ]:
def auto_shift_mask(
    mask,
    image,
    shift_range_y=(-10, 10),
    shift_range_x=(-10, 10),
    step_size=1,
    crop=None,
    method="minimize",
):
    """
    Automatically shifts a binary (beamstop) mask to the optimal position

    Parameter
    =========
    mask : array
        binary mask, e.g., beamstop mask
    image : array
        image partially covered by mask
    shift_range_y, _x : tupel
        Min and max limit for search area in y- and x-direction
    step_size : scalar
        step size of scan area
    crop : int
        additional cropping of arrays to speed up calculations
        Crops according to [crop:-crop,crop:-crop]
    method : string
        method used for optimization. 'minimize' or 'maximize' of scalarproduct metric <mask,image>

    Output
    ======
    optimized shift: tupel
        shift vector for optimized posistion
    mask_shifted: array
        mask shifted according to best shift vector
    overlap : array
        computed evaluation metric
    ======
    author: ck 2023
    """

    # Basic looping
    # Create set of arrays for shifting
    yshift = np.arange(shift_range_y[0], shift_range_y[1], step_size)
    xshift = np.arange(shift_range_x[0], shift_range_x[1], step_size)

    # Setup loss function to evaluate overlap
    overlap = np.zeros((yshift.shape[0], xshift.shape[0]))

    # Optinal cropping to reduce computation time
    if crop is not None:
        tmask, timage = mask[crop:-crop, crop:-crop], image[crop:-crop, crop:-crop]
    else:
        tmask, timage = mask.copy(), timage.copy()

    # Loop over all combinations of shifts
    for i, y in enumerate(yshift):
        for j, x in enumerate(xshift):
            # Shift mask to new position
            mask_shift = cci.shift_image(tmask, [y, x])

            # Calculate overlap
            overlap[i, j] = np.sum(mask_shift * timage)

    # Smoothing
    overlap = gaussian_filter(overlap,2)
    
    # Get best shift
    if method == "minimize":
        idx = np.unravel_index(
            np.argmin(
                overlap,
            ),
            overlap.shape,
        )
    elif method == "maximize":
        idx = np.unravel_index(
            np.argmax(
                overlap,
            ),
            overlap.shape,
        )

    # Output
    optimized_shift = (yshift[idx[0]], yshift[idx[1]])
    print(
        "Best mask overlap for shift: [%.2f,%.2f]"
        % (optimized_shift[0], optimized_shift[1])
    )

    # Shift
    mask_shifted = cci.shift_image(mask, optimized_shift)

    # Binarize mask (necessary for sub-px shift)
    mask_shifted[mask_shifted > 0.5] = 1
    mask_shifted[mask_shifted < 0.5] = 0

    return optimized_shift, mask_shifted, overlap


def multi_level_auto_shift_mask(
    mask,
    image,
    shift_range_y=(-10, 10),
    shift_range_x=(-10, 10),
    init_step_size=1,
    level = 1,
    crop=None,
    method="minimize",):
    
    """
    Automatically shifts a binary (beamstop) mask with multi-level optimization

    Parameter
    =========
    mask : array
        binary mask, e.g., beamstop mask
    image : array
        image partially covered by mask
    shift_range_y, _x : tupel
        Min and max limit for search area in y- and x-direction
    init_step_size : scalar
        initial step size of scan area
    level : int
        reduces step_size and range by 2**(level-1) per level step
    crop : int
        additional cropping of arrays to speed up calculations
        Crops according to [crop:-crop,crop:-crop]
    method : string
        method used for optimization. 'minimize' or 'maximize' of scalarproduct metric <mask,image>

    Output
    ======
    optimized shift: tupel
        shift vector for optimized posistion
    mask_shifted: array
        mask shifted according to best shift vector
    overlap : array
        computed evaluation metric
    ======
    author: ck 2023
    """

    #Prepare images and mask
    # Expand or contract Mask if necessary
    footprint = skimage.morphology.disk(6) #6 for binned
    mask_align = skimage.morphology.erosion(mask, footprint)
    #mask_align = skimage.morphology.dilation(mask, footprint)

    # Apply nr of levels
    optimized_shift = [0,0]
    for i in range(0,level):
        optimized_shift, _, overlap = auto_shift_mask(
            mask_align,
            image,
            shift_range_y=[optimized_shift[0] + shift_range_y[0]*0.5**i, optimized_shift[1]+shift_range_y[1]*0.5**i],
            shift_range_x=[optimized_shift[0] + shift_range_y[0]*0.5**i, optimized_shift[1]+shift_range_y[1]*0.5**i],
            step_size=init_step_size*0.5**i,
            crop=crop,
            method=method,
        )

    # Shift mask
    mask = cci.shift_image(mask,optimized_shift)
    mask[mask>0.5] = 1
    mask[mask<0.5] = 0

    return mask

In [ ]:
# Setup phase and propagation for cdi once
phase_cdi = 0
prop_dist_cdi = 0
dx = 0
dy = 0


def phase_retrieval(
    pos, neg, mask_pixel, supportmask, Startimage=None, Startgamma=None
):
    # Prepare Input holograms
    pos2 = pos.copy()
    neg2 = neg.copy()

    mi, _ = np.percentile(pos2[pos2 != 0], [1, 99.9])
    pos2 = pos2 - mi
    mi, _ = np.percentile(neg2[neg2 != 0], [1, 99.9])
    neg2 = neg2 - mi

    pos2[pos2 < 0] = 0
    neg2[neg2 < 0] = 0
    pos2 = pos2.astype(complex)
    neg2 = neg2.astype(complex)

    bsmask_p = mask_pixel.copy()
    bsmask_p[pos2 <= 2] = 1
    bsmask_n = mask_pixel.copy()
    bsmask_n[neg2 <= 2] = 1

    # Setup start image and startgamma
    if Startimage is None:
        Startimage = np.fft.fftshift(np.fft.ifft2(np.fft.ifftshift(supportmask)))
    else:
        Startimage = Startimage.copy()
    if Startgamma is None:
        Startgamma = np.ones(pos.shape) * 1e-6 * 2
        Startgamma[pos.shape[0] // 2, pos.shape[1] // 2] = 0.7
    else:
        Startgamma = Startgamma.copy()

    # Settings for phase retrieval reconstructions
    partial_coherence = True

    # Setup
    retrieved_p = np.zeros(pos2.shape, np.cdouble)
    retrieved_n = np.zeros(pos2.shape, np.cdouble)

    # Algorithms and Inital guess
    plt.rcParams["figure.dpi"] = 100
    print("CDI - larger mask")

    algorithm_list = ["mine", "mine", "mine"]
    Nit_list = [700, 50, 50]  # iterations for algorithm_list

    x = (np.sqrt(np.maximum(pos2, np.zeros(pos2.shape)))[mask_pixel == 0]).flatten()
    y = ((np.abs(Startimage))[mask_pixel == 0]).flatten()
    res = stats.linregress(x, y)
    Startimage -= res.intercept
    Startimage /= res.slope

    average_img = 30
    real_object = False  # always set to False

    if partial_coherence:
        RL_freq = 20
        RL_it = 50

        algorithm_list_pc = ["mine", "ER", "ER"]
        Nit_list_pc = [700, 50, 50]

    # Execute Phase retrieval
    start_time = time.time()
    for i in range(len(Nit_list) // 3):
        print("############ -   CDI")

        # Positive helicity - beta_mode="arctan"
        retrieved_p, Error_diff_p, Error_supp = PhR.PhaseRtrv_GPU(
            diffract=np.sqrt(np.maximum(pos2, np.zeros(pos2.shape))),
            mask=supportmask,
            mode=algorithm_list[3 * i],
            beta_zero=0.5,
            Nit=Nit_list[3 * i],
            beta_mode="arctan",
            plot_every=349,
            Phase=Startimage,
            seed=False,
            real_object=real_object,
            bsmask=bsmask_p,
            average_img=average_img,
            Fourier_last=True,
        )

        # Positive helicity - beta_mode="const"
        retrieved_p, Error_diff_p2, Error_supp = PhR.PhaseRtrv_GPU(
            diffract=np.sqrt(np.maximum(pos2, np.zeros(pos2.shape))),
            mask=supportmask,
            mode=algorithm_list[3 * i + 1],
            beta_zero=0.5,
            Nit=Nit_list[3 * i + 1],
            beta_mode="const",
            plot_every=24,
            Phase=retrieved_p,
            seed=False,
            real_object=real_object,
            bsmask=bsmask_p,
            average_img=average_img,
            Fourier_last=True,
        )

        # Negative helicity - beta_mode="arctan"
        retrieved_n, Error_diff_n2, Error_supp = PhR.PhaseRtrv_GPU(
            diffract=np.sqrt(np.maximum(neg2, np.zeros(neg2.shape))),
            mask=supportmask,
            mode=algorithm_list[3 * i + 2],
            beta_zero=0.5,
            Nit=Nit_list[3 * i + 2],
            beta_mode="const",
            plot_every=24,
            Phase=retrieved_p * np.sqrt(np.sum(neg2) / np.sum(pos2)),
            seed=False,
            real_object=real_object,
            bsmask=bsmask_n,
            average_img=average_img,
            Fourier_last=True,
        )

        print("--- %s seconds ---" % np.round((time.time() - start_time), 2))

        Startimage = retrieved_p.copy()

        # Partial coherence phase retrieval
        if partial_coherence:
            # CDI_PC
            print("############   -   CDI_pc")
            pos3 = (np.abs(retrieved_p) ** 2) * bsmask_p + np.maximum(
                pos2, np.zeros(pos2.shape)
            ) * (1 - bsmask_p)
            neg3 = (np.abs(retrieved_n) ** 2) * bsmask_n + np.maximum(
                neg2, np.zeros(neg2.shape)
            ) * (1 - bsmask_n)

            # retrieve pos image
            (
                retrieved_p_pc,
                Error_diff_p_pc,
                Error_supp,
                gamma_p,
            ) = PhR.PhaseRtrv_with_RL(
                diffract=np.sqrt(pos3),
                mask=supportmask,
                mode=algorithm_list_pc[3 * i],
                beta_zero=0.5,
                Nit=Nit_list_pc[3 * i],
                beta_mode="arctan",
                gamma=Startgamma,
                RL_freq=RL_freq,
                RL_it=RL_it,
                plot_every=349,
                Phase=Startimage,
                seed=False,
                real_object=False,
                bsmask=np.zeros(bsmask_p.shape),
                average_img=average_img,
                Fourier_last=True,
            )

            (
                retrieved_p_pc,
                Error_diff_p_pc2,
                Error_supp,
                gamma_p,
            ) = PhR.PhaseRtrv_with_RL(
                diffract=np.sqrt(pos3),
                mask=supportmask,
                mode=algorithm_list[3 * i + 1],
                beta_zero=0.5,
                Nit=Nit_list_pc[3 * i + 1],
                beta_mode="const",
                gamma=gamma_p,
                RL_freq=RL_freq,
                RL_it=RL_it,
                plot_every=24,
                Phase=retrieved_p_pc,
                real_object=False,
                bsmask=np.zeros(bsmask_p.shape),
                average_img=average_img,
                Fourier_last=True,
            )
            (
                retrieved_n_pc,
                Error_diff_n_pc2,
                Error_supp,
                gamma_n,
            ) = PhR.PhaseRtrv_with_RL(
                diffract=np.sqrt(neg3),
                mask=supportmask,
                mode=algorithm_list[3 * i + 2],
                beta_zero=0.5,
                Nit=Nit_list_pc[3 * i + 2],
                beta_mode="const",
                gamma=gamma_p,
                RL_freq=RL_freq,
                RL_it=RL_it,
                plot_every=24,
                Phase=retrieved_p_pc * np.sqrt(np.sum(neg2) / np.sum(pos2)),
                real_object=False,
                bsmask=np.zeros(bsmask_n.shape),
                average_img=average_img,
                Fourier_last=True,
            )

            print("--- %s seconds ---" % np.round((time.time() - start_time), 2))

            Startimage = retrieved_p_pc.copy()
            Startgamma = gamma_p.copy()

    print("Phase Retrieval Done!")

    return (
        retrieved_p,
        retrieved_n,
        retrieved_p_pc,
        retrieved_n_pc,
        bsmask_p,
        bsmask_n,
        gamma_p,
        gamma_n,
    )

In [ ]:
# Worker which performs complete fth reconstruction process
def worker(image, topo):
    # Centering
    shift_c = np.array(topo.shape) / 2 - center
    topo_c = cci.shift_image(topo, shift_c)
    im_c = cci.shift_image(image, shift_c)

    # Calc  shift
    shift = cci.image_registration(
        im_c[roi_im_reg],
        topo_c[roi_im_reg],
        method="dipy",
        #static_mask=mask_topo[roi_im_reg],
        #moving_mask=mask_im[roi_im_reg],
    )
    print("Shift: %s" % shift)

    # Shift and apply beamstop
    topo_c = cci.shift_image(topo, shift_c + shift)  # centered image

    # Create beamstop automatically
    mask_im, mask_topo, mask_pixel, mask_pixel_smooth = create_auto_beamstop(
        im_c, topo_c, mask_draw, use_bs, bs_param
    )

    # Get scaling factor and offset
    factor, offset = cci.dyn_factor(
        im_c * (1 - mask_pixel),
        topo_c * (1 - mask_pixel),
        method="correlation",
        print_out=True,
        plot=True,
    )

    # Apply beamstop to image
    im_b = im_c * mask_bs
    topo_b = topo_c * mask_bs  # centered image with beamstop

    # Calculate differences (magnetic) and sums (topographc) contrast holograms.
    # _c: centered, without beamstop, _b: centered, with beamstop
    diff_c = im_c / factor - topo_c - offset
    diff_b = im_b / factor - topo_b - offset
    sum_c = im_c / factor + topo_c - offset
    sum_b = im_b / factor + topo_b - offset

    # Do the reconstruction
    holo = diff_b.copy()
    recon = np.zeros(image.shape, dtype=np.complex_)

    # Reconstruct
    recon = cci.reconstruct(
        fth.propagate(holo, prop_dist * 1e-6, experimental_setup=experimental_setup)
        * np.exp(1j * phase)
    )

    # worker dictionary
    worker_dict = {}
    worker_dict["topo_c"] = topo_c
    worker_dict["im_c"] = im_c
    worker_dict["holo"] = holo
    worker_dict["sum"] = sum_b
    worker_dict["recon"] = recon
    worker_dict["factor"] = factor
    worker_dict["offset"] = offset
    worker_dict["shift"] = shift
    worker_dict["diff_c"] = diff_c
    worker_dict["diff_b"] = diff_b
    worker_dict["sum_c"] = sum_c
    worker_dict["sum_b"] = sum_b

    return worker_dict

In [ ]:
def optimize_phase_contrast(recon, supportmask, method="contrast", prefered_color=None):
    """
    Automatically shifts contrast of phase retrieval reconstruction into real part

    Parameter
    =========
    recon : complex array
        FTH/CDI reconstruction plane (Patterson map)
    supportmask : array
        Supportmask of Patterson map for phase retrieval
    method : string
        Choose method for phase optimization ("contrast","minima","maxima")
    prefered_color : string or None
        Shift contrast such that color of domains with largest are is white ("white"),
        black ("black") or non-specific (None)

    Output
    ======
    recon_optimized: complex array
        reconstruction with optimized contrast
    optimized_phase: complex scalar
        phase corresponding to optimized reconstruction
    ======
    author: ck 2023
    """

    # filter references from supportmask
    mask = supportmask.copy()
    mask = mask.astype(bool)
    mask = skimage.morphology.remove_small_objects(mask, min_size=200)
    mask = mask.astype(float)

    # Make object aperture smaller to minimize edge effects
    footprint = skimage.morphology.disk(4)
    mask = skimage.morphology.erosion(mask, footprint)

    # Gaussian filter to remove high intensity peaks
    reco = scipy.ndimage.gaussian_filter(recon, 1)

    # Different functions for optimization
    def contrast(phi, reco, tmask):
        temp = np.imag(reco * np.exp(1j * phi)) * tmask
        mi, ma = np.percentile(temp[temp != 0], [1, 99])
        contrast = ma - mi
        return contrast

    def minima(phi, reco, tmask):
        tmp = np.real(reco * np.exp(1j * phi)) * tmask
        minima, maxima = np.percentile(tmp[tmp != 0], [0.01, 99])
        return minima

    def maxima(phi, reco, tmask):
        tmp = np.real(reco * np.exp(1j * phi)) * tmask
        minima, maxima = np.percentile(tmp[tmp != 0], [0.01, 99])
        return maxima

    # Choose optimization method
    if method == "minima":
        optimized_phase = scipy.optimize.fminbound(
            minima, -np.pi, np.pi, args=(recon, mask), disp=False
        ).astype(float)
    elif method == "contrast":
        optimized_phase = scipy.optimize.fminbound(
            contrast, -np.pi, np.pi, args=(recon, mask), disp=False
        ).astype(float)
    elif method == "max":
        optimized_phase = scipy.optimize.fminbound(
            contrast, -np.pi, np.pi, args=(recon, mask), disp=False
        ).astype(float)

    # Calc optimized reconstruction
    recon_optimized = recon * np.exp(1j * optimized_phase)

    # Optional: Shift phase such that "background" are white or black
    mi_p, ma_p = np.percentile(np.real(recon_optimized[mask == 1]), [1, 99])
    mean = np.mean(np.real(recon_optimized[mask == 1]))
    if prefered_color == "white":
        # Make white domains the dominant domains
        if mean < (mi_p + ma_p) / 2:
            optimized_phase = optimized_phase + np.pi
            recon_optimized = recon * np.exp(1j * optimized_phase)
    elif prefered_color == "black":
        # Make black domains the dominant domains
        if mean > (mi_p + ma_p) / 2:
            optimized_phase = optimized_phase + np.pi
            recon_optimized = recon * np.exp(1j * optimized_phase)

    return recon_optimized, optimized_phase

In [ ]:
def get_title(im_id,topo_id):
    # Create title of reconstruction figure
    values = np.mean(np.array(load_data(im_id, "m_magnett_read")) * 1000)
    values = [np.round(values, 2)]
    temperature = load_collection(im_id,"cryob")
    title = "Image %d - %s @%.2f mT, %.2fK" % (im_id, topo_id, values[0],temperature)

    return title

# Experimental Details

In [ ]:
# Dict with most basic experimental parameter
experimental_setup = {
    "ccd_dist": 0.125+0.270,  # ccd to sample distance
    "px_size": 15e-6,  # pixel_size of camera
    "binning": 2,  # Camera binning
}

# Setup for azimuthal integrator
detector = Detector(
    experimental_setup["binning"] * experimental_setup["px_size"],
    experimental_setup["binning"] * experimental_setup["px_size"],
)

# General saving folder and log folder
folder_general = join("/DATA/export/cklose/2308_PETRA_Skyrmion_CCI","Analysis","FTH_CDI")
sup.create_folder(join(folder_general, "Logs"))
sup.create_folder(join(folder_general, "Topos"))
print("Output Folder: %s" % folder_general)

# Load images

Start by loading the images: image of interest (im), reference of charge scattering (topo), any kind of dark image (dark)

We estalished the following convention: Difference Hologram which contains only the magnetic scattering will be calculated according to:

$Diff = \frac{Image}{factor} - Topo$,

where the factor is used for intensity scaling. In Case that you recorded scans of the same magnetic state with both helicities, use the image with negative helicity as topo and the one with positive helicity as image

In [ ]:
# Define scan ids for each image
im_id = 394  # single helicity mode: image with magnetic contrast, double helicity: pos
topo_id = (
    395  # single helicity mode: image without magnetic contrast, double helicity: neg
)

# Load energy and add to experimental setup
experimental_setup["energy"] = load_energy(im_id)
experimental_setup["lambda"] = cci.photon_energy_wavelength(
    experimental_setup["energy"], input_unit="eV"
)

print("Image Id: %s" % im_id)
print("Topo Id: %s" % topo_id)



## Load image of interest

In [ ]:
# Load image
image, _ = load_processing(im_id)

# Plot
fig, ax = cimshow(image)
ax.set_title("Image")

## Load topo data set and average

In [ ]:
# Load topo
(
    topo,
    _,
) = load_processing(topo_id)

# Plot
fig, ax = cimshow(topo)
ax.set_title("Topo")

# Center holograms

* Find center of the hologram to get a well-defined q-space. 
* Create smooth mask for beamstop or overexposed areas in direct beam

## Basic widget to find center

Try to **align** the circles to the **center of the scattering pattern**. Care! Position of beamstop might be misleading and not represent the actual center of the hologram. 

In [ ]:
# Find center position via widget
c0, c1 = [521, 510]  # initial values
#c0, c1 = [1044, 1020]  # initial values
ic = interactive.InteractiveCenter(image, c0=c0, c1=c1)

In [ ]:
# Get center positions
center = [ic.c0, ic.c1]
print(f"Center:", center)
shift_c = np.array(image.shape) / 2 - center
print(f"Shift vector:", shift_c)

## Azimuthal integrator widget for finetuning
More of an "expert widget" which works very well for alignment if you have an Airy Pattern as a scattering image. PyFai transforms images from carthesian detector coordinate system into polar coordinate system with angle `phi` and radial distance `q` as axis (azimuthal transformation). The center of the coordinate system will be defined in the azimuthal integrator class and must not necessarily represents the center coordinates of your image array. If the center is set correctly, all rings of the Airy pattern will be transformed into a straight line in the I(q,chi)-plot as rings appear at a given q for all angles chi.

In [ ]:
# Setup azimuthal integrator for virtual geometry
ai = AzimuthalIntegrator(
    dist=experimental_setup["ccd_dist"],
    detector=detector,
    wavelength=experimental_setup["lambda"],
    poni1=center[0]
    * experimental_setup["px_size"]
    * experimental_setup["binning"],  # y (vertical)
    poni2=center[1]
    * experimental_setup["px_size"]
    * experimental_setup["binning"],  # x (horizontal)
)

In [ ]:
# Not the widget, just for double checking to find correct radial range for plotting
# Show some vertical lines at these qs
q_lines = [0.02, 0.04]

# Perform azimuthal transformation
I_t, q_t, phi_t = ai.integrate2d(
    image,
    500,  # number of points for phi
    radial_range=(0.01, 0.1),  # relevant q-range
    unit="q_nm^-1",
    correctSolidAngle=False,
)
# Combine in an xarray for plotting
az2d = xr.DataArray(I_t, dims=("phi", "q"), coords={"q": q_t, "phi": phi_t})

# Plot
fig, ax = plt.subplots()
mi, ma = np.percentile(I_t, [1, 95])
az2d.plot.imshow(ax=ax, vmin=mi, vmax=ma)
plt.title(f"Azimuthal integration")

# Add vertical lines to plot
for qt in q_lines:
    ax.axvline(qt, ymin=0, ymax=180, c="red")

In [ ]:
# The widget
aic = interactive.AzimuthalIntegrationCenter(
    np.log10(image),
    # image,
    ai,
    c0=center[0],
    c1=center[1],
    im_data_range=[1, 95],
    radial_range=(0.02, 0.08),
    circle_radius=20,
    qlines=[100, 110],
)

In [ ]:
# Get center positions from widget
center = [aic.c0, aic.c1]
print(f"Center:", center)

## Centering of image hologram

In [ ]:
# Apply to topo and image
shift_c = np.array(image.shape) / 2 - center
im_c = cci.shift_image(image, shift_c)
topo_c = cci.shift_image(topo, shift_c)  # centered image

# Image Registration

Relative drift between data holograms and their corresponding topo holograms is calculated by image registration algorithm. Necessary to get well-defined difference hologram. The reference is always the static background image (topo).

## Set Alignment ROI 

Set a region of interest (ROI) of reference (topo) use for image registration is performed. Don't select ROI which contains the beamstop as this leads to wrong results

How to use:
1. Zoom into the image and adjust your FOV until you are satisfied.
2. Save the axes coordinates.

In [ ]:
fig, ax = cimshow(im_c-topo_c)

In [ ]:
# Takes start and end of x and y axis
x1, x2 = ax.get_xlim()
y2, y1 = ax.get_ylim()
roi_im_reg = np.array([y1, y2, x1, x2]).astype(int)

#roi_im_reg = [507, 945, 516, 955]  # other sample
roi_im_reg = [275, 467, 272, 470]  # other sample
roi_im_reg = np.s_[roi_im_reg[0] : roi_im_reg[1], roi_im_reg[2] : roi_im_reg[3]]

print(f"Image registration roi:", roi_im_reg)

## Calculate drift of images

In [ ]:
shift = cci.image_registration(
    im_c[roi_im_reg],
    topo_c[roi_im_reg],
    method="dipy",
)
print(shift)

In [ ]:
shift = [0,0]

## Center and apply circular beamstop to topo image

In [ ]:
# Shift and apply beamstop
topo_c = cci.shift_image(topo, shift_c + shift)  # centered image

# Plot original and shifted holos
mi, ma = np.percentile(np.real(im_c[im_c != 0]), (0.1, 99))
fig, ax = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(12, 4))
ax[0].imshow(np.real(image), cmap="viridis", vmin=mi, vmax=ma)
ax[0].set_title("Uncentered image")
ax[1].imshow(np.real(im_c), cmap="viridis", vmin=mi, vmax=ma)
ax[1].set_title("Centered image")
mi, ma = np.percentile(im_c-topo_c,[.5,99.5])
ax[2].imshow(im_c-topo_c, cmap="viridis", vmin=mi, vmax=ma)
ax[2].set_title("im_c-topo_c")

# Add circles with different radi r
tmp = np.array(image.shape) / 2
for r in np.arange(50, 150, 25):
    ax[0].add_artist(plt.Circle((tmp[1], tmp[0]), r, fill=None, ec="red"))
    ax[1].add_artist(plt.Circle((tmp[1], tmp[0]), r, fill=None, ec="red"))
    ax[2].add_artist(plt.Circle((tmp[1], tmp[0]), r, fill=None, ec="red"))

# Create beamstops

We want to cover the beamstop with a smooth circle to cover its sharp edges as these would create ringing-like artifacts in the reconstruction plane. Make it only as large as necessary to keep as much information as possible.

## Circle beamstop

In [ ]:
# Set beamstop diameter and std for smoothing filter.
# Higher values mean stronger smoothing.
# If you have a very small beamstop you might need to reduce the smoothing value.
# Otherwise the beamstop center won't mask the direct beam with zeros
bs = interactive.InteractiveBeamstop(
    im_c, im_c.shape[0] / 2, im_c.shape[1] / 2, rBS=44, stdBS=4,
)

In [ ]:
# Take value from widget and create beamstop mask
bs_diam = bs.rBS
bs_smoothing = bs.stdBS
mask_bs = 1 - cci.circle_mask(
    topo.shape, np.array(topo.shape) / 2, bs.rBS, sigma=bs_smoothing
)

# Apply beamstop to image
im_b = im_c * mask_bs
topo_b = topo_c * mask_bs

# Plot image with beamstop and valid pixel mask
fig, ax = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(9, 3))
mi, ma = np.percentile(im_b, [0.1, 99.9])
ax[0].imshow(im_b, cmap="viridis", vmin=mi, vmax=ma)
ax[0].set_title("Masked image")

ax[1].imshow(mask_bs)
ax[1].set_title("Beamstop mask")

ax[2].imshow(1 - mask_bs)
ax[2].set_title("1 - Beamstop mask")

## Manual masking

In [ ]:
poly_mask = interactive.draw_polygon_mask(im_c)

In [ ]:
# Take poly coordinates and mask from widget
p_coord = poly_mask.coordinates
mask_draw = poly_mask.full_mask.astype(int)

print("Copy these coordinates into the 'load_poly_coordinates()' function:")
print(p_coord)

# Plot image with beamstop and valid pixel mask
fig, ax = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(9, 3))
mi, ma = np.percentile(im_c * (1 - mask_draw), [0.1, 99.9])
ax[0].imshow(im_c * (1 - mask_draw), cmap="viridis", vmin=mi, vmax=ma)
ax[0].set_title("Image * (1-mask_draw)")

mi, ma = np.percentile(im_c * mask_draw, [0.1, 99.9])
ax[1].imshow(im_c * mask_draw, vmin=mi, vmax=ma)
ax[1].set_title("Image * mask_draw")

ax[2].imshow(1 - mask_draw)
ax[2].set_title("1 - mask_draw")

In [ ]:
def load_poly_coordinates():
    """
    Dictionary that stores polygon corner coordinates of all drawn masks
    """

    # Setup dictonary
    mask_coordinates = dict()

    # Mask #1
    mask_coordinates["bs_small"] = [[(925.989313014395, 998.7406599510371), (797.5103561778718, 1004.2370859119578), (763.8447471672321, 995.9924469705767), (733.6144043821678, 1005.6111924021881), (260.07267008300744, 1028.132782654592), (-6.889330113719595, 1045.3819997176558), (-9.607990919821646, 1103.1535418473243), (384.5978259649745, 1082.763585801559), (726.7438719310169, 1066.0718779723165), (757.6612679611962, 1070.8812506881222), (807.8161548545983, 1055.76607929559), (943.8526973873875, 1054.39197280536), (986.6416451361885, 1089.1719039154077), (995.865316900066, 1112.8899170225213), (1000.4061264221758, 1342.985412380829), (1009.0419908484624, 1573.7807014097707), (1017.6654487908309, 1891.048467852913), (1016.9479952175003, 1940.0922371751922), (1006.5755826776376, 1940.9528653622822), (953.6999602651973, 1945.3629067545507), (957.5065573660545, 2053.558045894442), (1021.7728079434278, 2053.2891043152954), (1015.483922738999, 1977.1333369937422), (1078.731957691302, 1974.790806010689), (1064.283961377335, 1442.4223125802382), (1051.207352719271, 1103.5198276087256), (1069.654696247026, 1083.7548166861309), (1102.5963811180172, 1041.589460051262), (1090.7373745644604, 982.2944272834781), (1048.5720165333412, 934.8583878048696), (1056.478020902379, 843.939337560934), (1049.7028431303404, 675.373213310742), (1037.6888215077145, 441.56186942425256), (1021.3842224971681, 96.63212800981432), (1022.5114829926234, -6.834588443244456), (957.9457806454807, -10.787590627763393), (960.9970661640175, 109.63859245080062), (961.9260993383737, 186.74834592236203), (966.5712652101545, 327.96138842449864), (970.295635279982, 436.54068297309243), (977.6888793554441, 527.1079228975034), (976.7647238460113, 608.4336077275866), (980.0533120016796, 670.0072414421006), (986.6416489758778, 741.1612807634415), (989.2769837655571, 857.1160115093304), (998.5006555294347, 922.9993812513128), (970.6477739468762, 969.884423656203)]]
    mask_coordinates["bs_small_binned"] = [[(17.900266494115918, 519.9014527901415), (19.144865363833183, 551.431290822979), (192.83555456215583, 542.5808101548262), (297.0130923426968, 536.2195276657162), (359.24303582856015, 531.6559984767529), (368.7849604963925, 534.974928795999), (382.4755480632825, 535.8046613758104), (399.89993223932424, 530.8262658969413), (447.60955832603713, 528.2448719730263), (473.331268300194, 527.000273103309), (479.5542626487803, 534.4678663216126), (495.73404795510476, 546.4989887288796), (499.8827108541624, 565.9055153624472), (500.74643035245145, 565.8847534659697), (499.4678445642566, 570.4690445514105), (499.3732685901287, 609.8259298602966), (503.49275387709685, 729.2910031823731), (508.43110627901615, 865.3255894048384), (512.6291764482753, 1010.5788172612052), (481.56345719575756, 1010.578817261205), (481.56345719575756, 1026.111676887464), (511.36975539749756, 1026.9512909213158), (511.36975539749756, 1014.3570804135383), (539.9166325484598, 1014.3570804135383), (538.2374044807561, 882.1178700818748), (533.164208338346, 722.0335343323433), (529.0155454392885, 613.3385663770352), (526.6994057279082, 588.9974636684381), (526.4341534081307, 548.4811311864055), (546.7626016135127, 531.8864795901753), (550.0815319327588, 502.01610671696085), (537.635543235586, 483.34712367120187), (525.1895545384134, 472.9754664235579), (528.4701099393336, 455.43989356839137), (527.0315034142494, 405.0594979658785), (520.8085090656631, 261.1008953685812), (516.7268343486486, 171.09283721934466), (509.23167523872564, -5.043401863844878), (480.50023198402096, -4.210606407186759), (483.28615757785764, 90.59084593113596), (488.5352216353649, 236.48653967833388), (494.0434820910848, 354.91413947630974), (497.80837940463357, 419.45771502571546), (499.8827108541624, 459.284878856668), (494.9043153752933, 476.294396742804), (474.99073345981697, 495.7931123683745), (452.5879538049062, 499.5269089775263), (371.6890272732838, 503.2607055866781), (221.92229386975498, 511.32755027137085), (199.88828149055365, 511.88070470180025)]]
    mask_coordinates["bs_small_frame"] = [[(1862.0185550323479, -1.4342617134156512), (1861.9252083870479, 947.081063166574), (1861.7012739152497, 1469.593359916208), (1862.1455671735584, 1958.5554297887124), (1862.1455671735584, 2060.0790040354714), (1941.3801439264957, 2054.8144436055877), (1943.0734374940043, 1929.5107196099666), (1890.5509069127984, 1924.9896217349233), (1913.836496732065, 1108.1417332069568), (2050.2618772354645, 1103.073880151665), (2051.871698584416, 1045.7642401133173), (1913.4270625371628, 1045.7642401133173), (1920.3321882976975, 809.312128667904), (1940.291467940464, -2.3424235368384245)], [(-4.721815535160211, 1938.2802851388), (-3.066468771387285, 1959.5239019405526), (781.7292372187214, 1959.4533435946423), (779.2308445495314, 1956.3303527581547)], [(1941.2528662043205, 1935.4774150389783), (2049.7712972145796, 1939.6035911230188), (2049.5649884103773, 1960.440780347422), (1940.8402485959166, 1959.409236326412)]]
    mask_coordinates["bs_small_frame_binned"] = [[(-3.7233835208692767, -2.652325881343188), (-1.3490659189772458, 1026.8018950749463), (39.88908477819439, 1026.4270027958812), (40.51390630263684, 1015.2218905230094), (75.36453931273697, 1014.4317659729005), (423.63947609830893, 1013.5153626792488), (424.3892606564393, 1010.8911167257925), (15.756666941370653, 997.9365056940319), (19.311663741067036, 769.3342034116963), (25.768142399155366, 551.4384742958973), (27.64260379448136, 519.9475228544209), (34.99374430590498, 222.83883854375483), (40.88879768788911, -3.4021104394735744)]]
    mask_coordinates["other_binned"] = [[(375.4111742641943, 1007.4835760810058), (377.63930630639754, 1015.9504778413782), (481.0246330646287, 1019.9611155173441), (509.09909679638974, 1020.4067419257848), (510.43597602171167, 1014.6135986160563), (540.2929453872354, 1014.6135986160563), (541.1841982041167, 1021.2979947426661), (680.2196376375998, 1015.9504778413782), (679.3283848207186, 1007.4835760810058), (542.9667038378792, 1003.47293840504), (541.1841982041167, 1008.8204553063279), (484.14401792371325, 1010.1573345316498), (479.24212743086605, 1004.3641912219214)]]
    mask_coordinates["bs_small_frame_2"] = [[(73.13803227273192, 304.2749617603012), (54.05724374774849, 1036.9772783454634), (51.27597592125616, 1144.0884266788305), (27.538181262139176, 2001.4950183598755), (880.8052136142334, 2020.1877244796292), (946.3721317153299, 2026.284586834442), (905.2544958185244, 2028.9196109811617), (81.093751804872, 2028.272803631242), (77.60099720425899, 2052.139960068764), (-6.225113210453259, 2053.3042116023016), (-7.648087221326733, 993.3178458378007), (-5.589889104306053, -3.0874478741743587), (79.99418022422773, -4.8338251744808645)]]
    mask_coordinates["bs_small_5um"] = [[(53.80975749370184, 1107.3952320437968), (254.1660417793745, 1099.495880333609), (357.5757368945604, 1093.7508972716541), (385.58252932158996, 1092.3146515061655), (414.08123462695164, 1090.223186968721), (427.23365652076205, 1087.2877913269551), (447.3410972376038, 1090.1602828579325), (562.0145484722873, 1084.4782039067663), (733.3933909124834, 1072.3667925530053), (758.5276918085356, 1081.702390028682), (782.9438698218432, 1073.0849154357497), (810.2325393661285, 1067.339932373795), (881.8186151630628, 1064.0515969189128), (950.7584119065202, 1059.0247367397023), (960.812132264941, 1072.6690715118448), (988.8189246919706, 1093.4946351114309), (996.9371022847836, 1109.9941656911028), (1000.602079843482, 1148.1099323015667), (1001.173441572125, 1188.4633310972945), (999.8690843317422, 1219.9434924520565), (1008.3546703995685, 1465.6587638366125), (1019.710029212774, 1774.390923702081), (1029.6243155234567, 2055.484128044766), (1083.5431747049427, 2053.2619818912017), (1076.3023029018393, 1743.1006740509747), (1067.6240736124787, 1489.285536254233), (1061.8241811486819, 1316.0137488983053), (1059.974716294397, 1208.948559775961), (1054.8437477122193, 1183.293716865072), (1056.3224756699392, 1103.5483554698517), (1074.275547738548, 1089.9040206977093), (1093.6648655726453, 1068.3603342153788), (1098.7970016437619, 1029.3085817292401), (1094.3829884553898, 1000.85678323741), (1050.5774926079844, 962.7962704519596), (1046.1346551336253, 940.5804999965625), (1054.0340068438131, 909.7012160385555), (1054.7521297265575, 858.7144913637069), (1045.8072734215755, 584.9316965258525), (1038.9999988822556, 445.1743910173232), (1033.3226287755401, 287.6273705559689), (1029.774272458843, 212.4022166419889), (1019.5097296557678, -17.421842883846125), (963.2869044706213, -9.49298292183829), (965.4493208238962, 48.892258616583035), (969.0533480793542, 151.96743812268488), (973.7241149165386, 257.1755734909844), (977.2724712332358, 365.0456055185783), (981.5304988132725, 466.5285961761174), (986.4981987846406, 635.9823308672255), (991.4658976280166, 729.6589376280308), (998.1545221676472, 841.4335032681648), (983.7920645127601, 854.3597151575631), (993.1276619884367, 868.72217281245), (1001.0270136986245, 941.9707068523735), (991.691416222948, 969.2593763966588), (957.9396407339636, 1000.85678323741), (927.0603567759567, 1012.3467493613196), (831.3904264708616, 1015.7777834233457), (757.4237695481938, 1012.9052918923683), (744.4975576587955, 1018.6502749543231), (659.7590574949625, 1019.3683978370675), (616.6716759696096, 1019.1290242403572), (440.73156969724437, 1032.7733590125), (426.3691120423574, 1030.6189903642667), (386.1542306086739, 1034.2096047779885), (229.28426894092547, 1043.6249916136046), (53.34416266856023, 1050.8062204410483), (-15.83501090564664, 1048.6518531244785), (-12.244396491924903, 1110.4104210404923)], [(-6.336574907493798, -4.6772034563388445), (-3.9560769083365415, 1054.7411376096238), (-2.8065458245602457, 2050.8684126592934), (81.26952796433648, 2052.394104352098), (82.21369234580018, 2028.0731314107184), (1303.3156843144739, 2030.2833652860566), (1302.0835069496861, 2021.6581237325418), (32.46160852662814, 1998.3152069063863), (40.333853581431896, 1612.0275957010517), (52.58717307633992, 1131.1361396123257), (55.18843660147928, 1053.508960244836), (67.59767321816702, 575.5434252779708), (82.71159936273244, -3.4404232581412657)]]
    return mask_coordinates

In [ ]:
# Which drawn masks do you want to load?
polygon_names = ["bs_small_binned","bs_small_frame_binned"]
mask_draw = load_poly_masks(
    im_c.shape,
    polygon_names,
)

# Optimize overlap between beamstop mask and actual position
optimize_position = True

# Optimize position of drawn mask relative to target image
if optimize_position is True:
    mask_draw_im = multi_level_auto_shift_mask(
    mask_draw,
    im_c,
    shift_range_y=(-10, 10),
    shift_range_x=(-10, 10),
    init_step_size=2,
    level = 2,
    crop=300,
    method="minimize",)
    
    mask_draw_topo = multi_level_auto_shift_mask(
    mask_draw_im,
    topo_c,
    shift_range_y=(-10, 10),
    shift_range_x=(-10, 10),
    init_step_size=2,
    level = 2,
    crop=300,
    method="minimize",)

    # Combine both masks
    mask_draw = mask_draw_im + mask_draw_topo
    mask_draw[mask_draw >1] = 1

# Expand or contract Mask if necessary
footprint = skimage.morphology.disk(2)
#mask_draw = skimage.morphology.dilation(mask_draw, footprint) #increase
mask_draw = skimage.morphology.erosion(mask_draw, footprint) #decrease

# Shift mask is necessary
# mask_draw = cci.shift_image(mask_draw, [50, 49])

# Plot image with beamstop and valid pixel mask
fig, ax = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(9, 3))
mi, ma = np.percentile(im_c * (1 - mask_draw), [0.1, 99.9])
ax[0].imshow(im_c * (1 - mask_draw), cmap="viridis", vmin=mi, vmax=ma)
ax[0].set_title("Image * (1-mask_draw)")

mi, ma = np.percentile(im_c * mask_draw, [0.1, 99.9])
ax[1].imshow(im_c * mask_draw, vmin=mi, vmax=ma)
ax[1].set_title("Image * mask_draw")

ax[2].imshow(1 - mask_draw)
ax[2].set_title("1 - mask_draw")

## Automated Beamstop detection

In [ ]:
# Initial parameter for automatic beamstop detection
thres = 70
radi_thres = 210
expand = 8
method_bs = "gradient"
auto_bs = interactive.InteractiveAutoBeamstop(
    im_c, thres, radi_thres, expand, method=method_bs
)

In [ ]:
# Initial parameter for automatic beamstop detection
thres = 50
radi_thres = 100
expand = 8
auto_bs_otsu = interactive.InteractiveAutoBeamstop(
    topo_c, thres, radi_thres, expand, method="otsu"
)

## Combine all beamstops

In [ ]:
def create_auto_beamstop(im_c, topo_c, mask_draw, use_bs, bs_param):
    # Use drawn beamstop?
    if use_bs["use_draw"] is True:
        # Use drawn beamstop only for specific q-range
        tmp = cci.circle_mask(
            im_c.shape, np.array(im_c.shape) / 2, bs_param["bs_radius"], sigma=None
        )

        # Load drawn beamstop0
        mask_draw = load_poly_masks(
        im_c.shape,
        polygon_names,
        )
        
        # Optimize position of drawn mask relative to target image
        if optimize_position is True:
            # For im_c
            mask_im = multi_level_auto_shift_mask(
            mask_draw,
            im_c,
            shift_range_y=(-10, 10),
            shift_range_x=(-10, 10),
            init_step_size=2,
            level = 2,
            crop=300,
            method="minimize",)

            # For topo_c
            mask_topo = multi_level_auto_shift_mask(
            mask_im,
            topo_c,
            shift_range_y=(-10, 10),
            shift_range_x=(-10, 10),
            init_step_size=2,
            level = 2,
            crop=300,
            method="minimize",)
    else:
        tmp = cci.circle_mask(
            im_c.shape, np.array(im_c.shape) / 2, bs_param["rBS"], sigma=None
        )
        mask_im = tmp.copy()
        mask_topo = tmp.copy()

    
    # Use automatically determined beamstop mask?
    if use_bs["use_bs_auto"] is True:
        tmp = cci.circle_mask(
            im_c.shape, np.array(im_c.shape) / 2, bs_param["bs_radius"], sigma=None
        )
        mask_im = mask_im*(1-tmp)+tmp*automated_beamstop(
            im_c,
            bs_param["bs_thres"],
            bs_param["bs_radius"],
            bs_param["bs_expand"],
            method=bs_param["method"],
        )
        mask_topo = mask_topo*(1-tmp) + tmp*automated_beamstop(
            topo_c,
            bs_param["bs_thres"],
            bs_param["bs_radius"],
            bs_param["bs_expand"],
            method=bs_param["method"],
        )

    # Use otsu mask as center?
    if use_bs["use_otsu"] is True:
        tmp = cci.circle_mask(
            im_c.shape, np.array(im_c.shape) / 2, auto_bs_otsu.radius, sigma=None
        )
        mask_im = tmp * automated_beamstop_center(
            im_c, auto_bs_otsu.expand
        ) + (1 - tmp) * mask_im 
        mask_topo = tmp * automated_beamstop_center(
            topo_c, auto_bs_otsu.expand
        ) + (1 - tmp) * mask_topo

    # Saturated pixel
    mask_im = mask_im + (im_c > 65000)
    mask_topo = mask_topo + (topo_c > 65000)

    # normalize
    mask_im[mask_im > 1] = 1
    mask_topo[mask_topo > 1] = 1

    # Combine both
    mask_pixel = mask_im + mask_topo
    mask_pixel[mask_pixel > 1] = 1

    # Additional masking
    # mask_pixel[1000:1500, 450:750] = 1

    # Sharp and smooth mask
    footprint = skimage.morphology.disk(6)
    mask_pixel_smooth = skimage.morphology.dilation(mask_pixel, footprint)
    mask_pixel_smooth = gaussian_filter(mask_pixel_smooth, 2)

    return mask_im, mask_topo, mask_pixel, mask_pixel_smooth

In [ ]:
# Parameter for automatically determined beamstop
use_bs = {"use_bs_auto": True, "use_otsu": True, "use_draw": True}
bs_param = {
    "bs_thres": auto_bs.thres,
    "bs_radius": auto_bs.radius,
    "bs_expand": auto_bs.expand,
    "method": method_bs,
    "rBS": bs.rBS,
    "stdBS": bs.stdBS,
}

# Create beamstop automatically
mask_im, mask_topo, mask_pixel, mask_pixel_smooth = create_auto_beamstop(
    im_c, topo_c, mask_draw, use_bs, bs_param
)


# Plot both
fig, ax = plt.subplots(2, 4, figsize=(10, 5), sharex=True, sharey=True)
mi, ma = np.percentile(im_c, [1, 99.9])
ax[0, 0].imshow(im_c, vmin=mi, vmax=ma)
ax[0, 0].set_title("Image")
mi, ma = np.percentile(im_c * mask_im, [1, 99.99])
ax[0, 1].imshow(im_c * mask_im, vmin=mi, vmax=ma)
ax[0, 1].set_title("Image*mask")
mi, ma = np.percentile(im_c * (1 - mask_im), [0.1, 99.9])
ax[0, 2].imshow(im_c * (1 - mask_im), vmin=mi, vmax=ma)
ax[0, 2].set_title("Image*(1-mask)")
ax[0, 3].imshow(mask_pixel_smooth)
ax[0, 3].set_title("Combined Mask")

mi, ma = np.percentile(topo_c, [1, 99.9])
ax[1, 0].imshow(topo_c, vmin=mi, vmax=ma)
ax[1, 0].set_title("Topo")
mi, ma = np.percentile(topo_c * mask_im, [1, 99.99])
ax[1, 1].imshow(topo_c * mask_topo, vmin=mi, vmax=ma)
ax[1, 1].set_title("Topo*mask")
mi, ma = np.percentile(topo_c * (1 - mask_topo), [0.1, 99.9])
ax[1, 2].imshow(topo_c * (1 - mask_topo), vmin=mi, vmax=ma)
ax[1, 2].set_title("topo*(1-mask)")
mi, ma = np.percentile((im_c - topo_c) * (1 - mask_pixel), [0.1, 99.9])
ax[1, 3].imshow((im_c - topo_c) * (1 - mask_pixel), vmin=mi, vmax=ma)
ax[1, 3].set_title("Image-Topo")

# Calculate difference holograms

You can see the reconstrution of the magnetization only after subtracting the large background that you get from the diffraction on the circular object aperture (Airy Pattern). This might require a scaling factor to correct intensity changes between the hologram and the topo. Scaling factor will be determined automatically by a linear fit. If the fit seems off, there might be an issue with the data

In [ ]:
# Get scaling factor and offset
factor, offset = cci.dyn_factor(
    im_c * (1 - mask_pixel),
    topo_c * (1 - mask_pixel),
    method="correlation",
    print_out=True,
    plot=True,
)

# Calculate differences (magnetic) and sums (topographc) contrast holograms.
# _c: centered, without beamstop, _b: centered, with beamstop
diff_c = im_c / factor - topo_c - offset
diff_b = im_b / factor - topo_b - offset
sum_c = im_c / factor + topo_c - offset
sum_b = im_b / factor + topo_b - offset

In [ ]:
# Plot an example of the difference or sum hologram
fig, ax = cimshow(diff_b * (1 - mask_pixel))
ax.set_title(f" Diff Id %d" % im_id)

# fig, ax = cimshow(sum_b)
# ax.set_title(f" Sum Id %d" % im_id)

# Reconstruct Diff Holos (FTH)

Reconstruct the hologramm.
1. Chose a region of interest (ROI) which means selecting one reconstruction from the rconstruction plane.
2. Propagate the image and shift the phase for maximal contrast and sharpness in your ROI
3. Optional finetuning with a widget

## Set Patterson Map ROI

Choose the reconstructions as the ROI.

1. Zoom into the image and adjust your FOV until you are satisfied.
2. Save the axes coordinates.

In [ ]:
# Choose contrast mode
# diff_b: magnetic contrast only
tmp = np.real(cci.reconstruct(diff_b* (1 - mask_pixel_smooth)))

# sum_b: topographic contrast only
# tmp = np.real(cci.reconstruct(sum_b * (1 - mask_pixel_smooth)))
fig, ax = cimshow(tmp, cmap="gray")

In [ ]:
# Execute to get roi
x1, x2 = ax.get_xlim()
y2, y1 = ax.get_ylim()
roi = np.array([y1, y2, x1, x2]).astype(int)  # ystart, ystop, xstart, xstop
#roi = [876, 1200, 831,1213]
#roi = [513, 654, 235,371]
#roi = [ 526, 1521 , 552, 1499]
#roi = [517, 590, 350, 423]
#roi = [550, 787, 901, 1155]
roi = [345 ,414, 474 ,548]
roi_s = np.s_[roi[0] : roi[1], roi[2] : roi[3]]
print(f"Roi Reco:{roi}")

## Tune propagation and phase
Focus the image by tuning the propagation distance. This really works like focussing in a microscope.
Phase slider will move contrast between real and imaginary part. Usually we use the phase which maximizes the contrast in the real part

In [ ]:
# Widget
slider_prop, slider_phase, button = reco.propagate(
    diff_b * (1 - mask_pixel_smooth),
    roi_s,
    phase=0,
    prop_dist=0,
    experimental_setup=experimental_setup,
    scale=(1, 99),
)

In [ ]:
# Read prop dist and phase from widget
prop_dist = slider_prop.value
phase = slider_phase.value

print(f"Propagation distance: %0.2f" % prop_dist)
print(f"Phase: %0.2f" % phase)

## Ugly Widget for finetuning of shift and factor
You can try to find a better scaling factor and optimize the relative shift between image and topo hologram with this widget. Just execute, find your values and then execute next cell to update the values. 

In [ ]:
# Ta-ke initial values
temp_factor = factor
temp_shift = shift.copy()
temp_phase = phase

# Open figure
fig, [ax, ax2] = plt.subplots(1, 2, figsize=(10, 5))

# Widget
layout = ipywidgets.Layout(width="100%")
opts = dict(layout=layout)


@ipywidgets.interact(
    centery=(center[0] - 10, center[0] + 10, 0.5),
    centerx=(center[1] - 10, center[1] + 10, 0.5),
    tfactor=(
        0.6,
        1.4,
        0.0025,
    ),
    xshift=(-4, 4, 0.01),
    yshift=(-4, 4, 0.01),
    tphase=(-2 * np.pi, 2 * np.pi, 0.05),
    options=opts,
)
def update(
    centery=center[0],
    centerx=center[1],
    tfactor=factor,
    yshift=shift[0],
    xshift=shift[1],
    tphase=phase,
):  # initial values
    global temp_shift, temp_factor, temp_center, temp_diff, temp_sum, temp_phase  # ugly writing as global variable

    # Couple widget only and global variables
    temp_shift = [yshift, xshift]
    temp_factor = tfactor
    temp_center = [centery, centerx]
    temp_phase = tphase

    # Calc diff and sum holo
    shifted = cci.shift_image(im_c, temp_shift)
    temp_diff = shifted - temp_factor * topo_c
    temp_sum = shifted + temp_factor * topo_c

    temp_diff = cci.shift_image(
        temp_diff, -np.array([centery - center[0], centerx - center[1]])
    )
    temp_sum = cci.shift_image(
        temp_sum, -np.array([centery - center[0], centerx - center[1]])
    )

    temp_diff = temp_diff * mask_bs * (1 - mask_pixel_smooth)

    # Reconstruction
    recon = cci.reconstruct(
        fth.propagate(
            temp_diff,
            prop_dist * 1e-6,
            experimental_setup=experimental_setup,
        )
        * np.exp(1j * temp_phase)
    )[roi_s]

    # Plots
    vmin, vmax = np.percentile(np.real(recon), (1, 99))
    ax.imshow(np.real(recon), vmin=vmin, vmax=vmax, cmap="gray")
    vmin, vmax = np.percentile(np.imag(recon), (1, 99))
    ax2.imshow(np.imag(recon), vmin=vmin, vmax=vmax, cmap="gray")
    # vmin, vmax = np.percentile(temp_diff * mask_bs * (1 - mask_pixel), (0.1, 99.9))
    # ax2.imshow(temp_diff * mask_bs, vmin=vmin, vmax=vmax)

In [ ]:
# Copy values from widget
factor = temp_factor
shift = temp_shift
phase = temp_phase
center = temp_center.copy()
diff_c = temp_diff.copy()
sum_c = temp_sum.copy()
diff_b = diff_c * mask_bs
sum_b = sum_c * mask_bs

## Save reconstruction

Save png files of the images and a h5 file for all important variables

In [ ]:
def plot_recon(recon, title):
    # Plot
    fig, ax = plt.subplots(1, 2, figsize=(10, 4))
    fig.suptitle(title)

    vmin, vmax = np.percentile(np.real(recon), (1, 99))
    t_im1 = ax[0].imshow(np.real(recon), vmin=vmin, vmax=vmax, cmap="gray")
    ax[0].set_title("Real")
    plt.colorbar(t_im1, ax=ax[0], aspect=50)

    vmin, vmax = np.percentile(np.imag(recon), (1, 99))
    t_im2 = ax[1].imshow(np.imag(recon), vmin=vmin, vmax=vmax, cmap="gray")
    ax[1].set_title("Imag")
    plt.colorbar(t_im2, ax=ax[1], aspect=50)

In [ ]:
# Saves only real and imaginary part
holo = diff_b * (1 - mask_pixel_smooth)
#holo = diff_b.copy()# * (1 - mask_pixel_smooth)
recon = np.zeros(image.shape, dtype=np.complex_)

# Reconstruct
recon = fth.reconstruct(
    fth.propagate(holo, prop_dist * 1e-6, experimental_setup=experimental_setup)
    * np.exp(1j * phase)
)

# Create plot
plot_recon(recon[roi_s], get_title(im_id,topo_id))

# Save images
fname = join(
    folder_general,
    "Recon_ImId_%04d_RefId_%04d_%s.png" % (im_id, topo_id, USER),
)
print("Saving: %s" % fname)
plt.savefig(fname, bbox_inches="tight", transparent=False)

# Save hdf5 file
save_fth_h5()

In [ ]:
# Closes all existing plots
plt.close("all")

# CDI Reconstruction

## Create set of pos and neg helicity holograms

In [ ]:
# Copy values from FTH reco
pos = (sum_c + diff_c) / 2
neg = (sum_c - diff_c) / 2

## Create hologram mask for phase retrieval
``mask_pixel[px,px] == 1`` means invalid, i.e., will be replaced by phase retrieval

In [ ]:
# Initial parameter for automatic beamstop detection
auto_bs = interactive.InteractiveAutoBeamstop(
    neg, auto_bs.thres, auto_bs.radius, auto_bs.expand, method=method_bs
)

In [ ]:
# Initial parameter for automatic beamstop detection
auto_bs_otsu = interactive.InteractiveAutoBeamstop(
    neg, thres, auto_bs_otsu.radius, auto_bs_otsu.expand, method="otsu"
)

In [ ]:
# Parameter for automatically determined beamstop
use_bs = {"use_bs_auto": True, "use_otsu": True, "use_draw": True}
bs_param = {
    "bs_thres": auto_bs.thres,
    "bs_radius": auto_bs.radius,
    "bs_expand": auto_bs.expand,
    "method": method_bs,
    "rBs": bs.rBS,
    "stdBS": bs.stdBS,
}

# Create beamstop automatically
mask_im, mask_topo, mask_pixel, mask_pixel_smooth = create_auto_beamstop(
    pos, neg, mask_draw, use_bs, bs_param
)

# More masking
#mask_pixel[1956:1962,780:1120] = 1
#mask_pixel[:,1859:1880] = 1

# Plot both
fig, ax = plt.subplots(2, 4, figsize=(12, 6), sharex=True, sharey=True)
mi, ma = np.percentile(pos, [1, 99.9])
ax[0, 0].imshow(pos, vmin=mi, vmax=ma)
ax[0, 0].set_title("Image")
mi, ma = np.percentile(pos * mask_im, [0.1, 99.99])
ax[0, 1].imshow(pos * mask_im, vmin=mi, vmax=ma)
ax[0, 1].set_title("Image*mask")
mi, ma = np.percentile(pos * (1 - mask_im), [0.1, 99.9])
ax[0, 2].imshow(pos * (1 - mask_im), vmin=mi, vmax=ma)
ax[0, 2].set_title("Image*(1-mask)")
ax[0, 3].imshow(mask_pixel_smooth)
ax[0, 3].set_title("Combined Smoothed Mask")

mi, ma = np.percentile(neg, [1, 99.9])
ax[1, 0].imshow(neg, vmin=mi, vmax=ma)
ax[1, 0].set_title("Neg")
mi, ma = np.percentile(neg * mask_topo, [0.1, 99.99])
ax[1, 1].imshow(neg * mask_topo, vmin=mi, vmax=ma)
ax[1, 1].set_title("Neg")
mi, ma = np.percentile(neg * (1 - mask_topo), [0.1, 99.9])
ax[1, 2].imshow(neg * (1 - mask_topo), vmin=mi, vmax=ma)
ax[1, 2].set_title("Neg*(1-mask)")
mi, ma = np.percentile((pos - neg) * (1 - mask_pixel), [0.1, 99.9])
ax[1, 3].imshow((pos - neg) * (1 - mask_pixel), vmin=mi, vmax=ma)
ax[1, 3].set_title("Pos-Neg")

## Draw support mask

## Execute if you want to create a new support mask

In [ ]:
# How many references do you have?
nr_ref = 5

# Setup coordinates (nr_ref + 1 coordinates, as there is always the object aperture)
support_coordinates = [
    [pos.shape[-2] // 2, pos.shape[-1] // 2, 7] for k in range(nr_ref + 1)
]

## Execute if you want to load an existing support mask

In [ ]:
def get_supportmask_coordinates(sample):
    """
    Dictionary that stores coordinates of circular support mask apertures
    """

    # Setup dictonary
    support_coord = dict()

    # coordinates
    support_coord["s2305e-C5"] = [(916.5, 946.0, 32.0), (875.0, 1071.0, 4.0), (783.5, 944.5, 4.0), (876.0, 819.0, 4.0), (1025.0, 868.0, 4.0), (1024.0, 1024.0, 4.0)]
    support_coord["s2305e-C5_binned"] = [(553.0, 388.0, 32.0), (421.5, 386.5, 4.0), (512.5, 262.5, 4.0), (659.5, 310.5, 4.0), (659.5, 464.5, 4.0), (512.0, 512.0, 4.0)]
    support_coord["s2305e-D5"] = [(581.5, 305.0, 64.5), (361.5, 303.0, 4.0), (514.0, 96.0, 4.0), (759.0, 177.0, 4.0), (758.0, 434.0, 5.0), (512.0, 512.0, 5.0)]
    support_coord["s2305e-E5"] = [(743.0, 816.0, 109.0), (389.5, 814.5, 4.0), (633.0, 483.0, 4.0), (1025.5, 612.5, 4.0), (630.0, 1149.0, 4.5), (1024.0, 1024.0, 4.0)]
    return support_coord[sample]

### Create new support mask

In [ ]:
# Which supportmask to load? ("s2306a-C1", "s2308a-B1", ...)
sample = "s2305e-C5_binned"

# Get coordinates and create supportmask
support_coordinates = get_supportmask_coordinates(sample)

In [ ]:
ds = interactive.InteractiveCircleCoordinates(
    np.real(
        fth.reconstruct((pos) * mask_bs * np.exp(1j * phase) * (1 - mask_pixel_smooth))
    ),
    len(support_coordinates),
    coordinates=support_coordinates,
)

In [ ]:
# Take coordinates of circles from widget
support_coordinates = ds.get_mask()

# Create supportmask
supportmask = create_supportmask(support_coordinates, pos.shape)

# What to plot?
tmp = np.real(
    fth.reconstruct(pos * mask_bs * np.exp(1j * phase) * (1 - mask_pixel_smooth))
)

# Plot supportmask as overlay
fig, ax = plt.subplots(figsize=(6, 6))
mi, ma = np.percentile(tmp, (1, 99))
ax.imshow(tmp, vmin=mi, vmax=ma, cmap="gray")
ax.imshow(supportmask, alpha=0.3, cmap="binary")
ax.set_title("Image with overlayed mask")

### Take Roi

In [ ]:
roi_cdi = interactive.axis_to_roi(ax)
roi_cdi = [517,588,349,428] #C5 binned
#roi_cdi = [877, 954, 906, 983]  # Single
#roi_cdi = [349, 774, 74, 527]  # All
#roi_cdi = [509, 653, 235, 374]  # Single
#roi_cdi = [615, 867, 692, 936]  # Single 5um
#roi_cdi = [515,592,349,425]
roi_cdi = np.s_[roi_cdi[0] : roi_cdi[1], roi_cdi[2] : roi_cdi[3]]
print("Sliced roi:", roi_cdi)

## Do Phase Retrieval

In [ ]:
(
    retrieved_p,
    retrieved_n,
    retrieved_p_pc,
    retrieved_n_pc,
    bsmask_p,
    bsmask_n,
    gamma_p,
    gamma_n,
) = phase_retrieval(pos, neg, mask_pixel, supportmask, Startimage=None, Startgamma=None)

## Reconstruct images

In [ ]:
# New beamstop for CDI recos as phase retrievel of low-q might be poor
use_bs_reco = True
bs_diam_cdi = 10  # 30#50

# Create beamstop
if use_bs_reco is True:
    mask_bs_cdi = 1 - cci.circle_mask(
        topo.shape, np.array(topo.shape) / 2, bs_diam_cdi, sigma=4
    )
    mask_bs_cdi = 1 - mask_pixel_smooth.copy()
elif use_bs_reco is False:
    mask_bs_cdi = np.ones(pos.shape)  # if you don't want a beamstop

# Get Recos
# Positive helicity
p = fth.reconstructCDI(
    fth.propagate(
        retrieved_p * mask_bs_cdi,
        prop_dist_cdi * 1e-6,
        experimental_setup=experimental_setup,
    )
)

# Negative helicity
n = fth.reconstructCDI(
    fth.propagate(
        retrieved_n * mask_bs_cdi,
        prop_dist_cdi * 1e-6,
        experimental_setup=experimental_setup,
    )
)
# p = fth.sub_pixel_centering(p, dx, dy)
# n = fth.sub_pixel_centering(n, dx, dy)


# Get Recos partial coherence
# Positiv partial coherence
p_pc = fth.reconstructCDI(
    fth.propagate(
        retrieved_p_pc * mask_bs_cdi,
        prop_dist_cdi * 1e-6,
        experimental_setup=experimental_setup,
    )
)

# Negative partial coherence
n_pc = fth.reconstructCDI(
    fth.propagate(
        retrieved_n_pc * mask_bs_cdi,
        prop_dist_cdi * 1e-6,
        experimental_setup=experimental_setup,
    )
)
# p_pc = fth.sub_pixel_centering(p_pc, dx, dy)
# n_pc = fth.sub_pixel_centering(n_pc, dx, dy)

# optimize phase
# recon = p - n
recon = p_pc - n_pc  # / (p_pc + n_pc)
_, phase_cdi = optimize_phase_contrast(
    recon, supportmask, method="contrast", prefered_color="white"
)

# Plotting
mode = "-"
slider_prop, slider_phase, slider_dx, slider_dy = rec.focusCDI(
    retrieved_p_pc * mask_bs_cdi,
    retrieved_n_pc * mask_bs_cdi,
    roi_cdi,
    mask=supportmask,
    phase=phase_cdi,
    dx=dx,
    dy=dy,
    prop_dist=prop_dist_cdi,
    experimental_setup=experimental_setup,
    operation=mode,
    max_prop_dist=10,
    scale=(.1, 99.9),
)

In [ ]:
# Get phase from slider
phase_cdi = slider_phase.value
prop_dist_cdi = slider_prop.value

# Apply phase
p = p * np.exp(1j * phase_cdi)
n = n * np.exp(1j * phase_cdi)
p_pc = p_pc * np.exp(1j * phase_cdi)
n_pc = n_pc * np.exp(1j * phase_cdi)

print("Phase CDI: %s" % phase_cdi)
print("Prop_dist: %s" % prop_dist_cdi)

In [ ]:
# Confirm that offset subtraction works, i.e., only small fraction of hologram is actually masked
fig, ax = plt.subplots(2, 2, figsize=(8, 8), sharex=True, sharey=True)
tmp = np.abs(retrieved_p_pc * mask_bs_cdi)
mi, ma = np.percentile(tmp, [0.1, 99.9])
ax[0, 0].imshow(tmp, vmin=mi, vmax=ma)
ax[0, 0].set_title("Pos holo with bs")

tmp = np.abs(retrieved_n_pc)
mi, ma = np.percentile(tmp, [0.1, 99.9])
ax[0, 1].imshow(tmp, vmin=mi, vmax=ma)
ax[0, 1].set_title("Neg holo")
ax[1, 0].imshow(bsmask_p)
ax[1, 0].set_title("Pos holo mask")
ax[1, 1].imshow(bsmask_n)
ax[1, 1].set_title("Neg holo mask")

## Save

In [ ]:
def plot_recon(recon, title):
    # Plot
    fig, ax = plt.subplots(1, 2, figsize=(10, 4))
    fig.suptitle(title)

    vmin, vmax = np.percentile(np.real(recon), (.1, 99.9))
    t_im1 = ax[0].imshow(np.real(recon), vmin=vmin, vmax=vmax, cmap="gray")
    ax[0].set_title("Real")
    plt.colorbar(t_im1, ax=ax[0], aspect=50)

    vmin, vmax = np.percentile(np.imag(recon), (.1, 99.9))
    t_im2 = ax[1].imshow(np.imag(recon), vmin=vmin, vmax=vmax, cmap="gray")
    ax[1].set_title("Imag")
    plt.colorbar(t_im2, ax=ax[1], aspect=50)

In [ ]:
# Saves only real and imaginary part
recon_all = p - n
recon_all = p_pc - n_pc  # / (p_pc + n_pc)
recon = recon_all.copy()

# Plot
plot_recon(recon[roi_cdi], get_title(im_id,topo_id))

# Save images
fname = join(
    folder_general,
    "Recon_ImId_%04d_RefId_%s_cdi_%s.png" % (im_id, topo_id, USER),
)
print("Saving: %s" % fname)
plt.savefig(fname, bbox_inches="tight", transparent=False)

# Save h5
save_cdi_h5()

# Batch processing CDI

## Define Scan Ids

In [ ]:
# Load support mask of which sample?
sample = "s2305e-C5_binned"

In [ ]:
# Define the sets for reconstructions. You can make a list or use np.arange
# im_id_set should always have ids of positive helicity holograms,
# topo_id_set those of negative helicity or a proper topo

im_id_set = np.arange(384, 385 + 1)
topo_id_set = 383 * np.ones(len(im_id_set), dtype=int)

# In case of single helicity reconstructions, adapt the helicity
# for contrast inversion
helicity = 1 * np.ones(len(im_id_set), dtype=int)  # [1,-1]

# Do cdi?
do_cdi = True

# Save fth reco?
save_fth = True

print("Dynamics Set:  %s" % im_id_set)
print("Reference Set: %s" % topo_id_set)
print("Helicity: %s" % helicity)

## Execute Stack Reconstruction

In [ ]:
# Ugly Automatic processing of image stacks
recons_name = []  # for gifs
for it, im_id in enumerate(im_id_set):
    # Load images
    image, _ = load_processing(im_id)

    # Get topo
    # Do you want to construct topo holo from two helicity images
    try:
        len(topo_id_set[it])
    except:
        # Usual case
        # Get also topo & dark id from list of sets
        topo_id = topo_id_set[it]

        # Load data
        print(f"Loading imageId: %04d, topoId: %04d" % (im_id, topo_id))
        topo, _ = load_processing(topo_id)

        # Process images
        worker_dict = worker(image, topo)

        # Save topo hologram
        save_topo_holo(worker_dict["sum_c"], im_id, topo_id)
    else:
        print("Using Topo from sum of two helicity holograms")
        pos_id = topo_id_set[it][0]
        neg_id = topo_id_set[it][1]
        topo_id = topo_id_set[it]

        try:
            topo = load_topo_holo(pos_id, neg_id) / 2
        except:
            topo = load_topo_holo(neg_id, pos_id) / 2
        topo = cci.shift_image(
            topo, -shift_c
        )  # shift out of center so you don't need to change the worker

        # Process images
        worker_dict = worker(image, topo)

    # Reconstruct
    recon = fth.reconstruct(
        fth.propagate(
            worker_dict["holo"], prop_dist * 1e-6, experimental_setup=experimental_setup
        )
        * np.exp(1j * phase)
    )

    # Plot
    plot_recon(recon[roi_cdi], get_title(im_id,topo_id))

    # Save images
    if save_fth is True:
        fname = join(
            folder_general,
            "Recon_ImId_%04d_RefId_%s_fth_diff_stack_%s.png" % (im_id, topo_id, USER),
        )
        print("Saving: %s" % fname)
        plt.savefig(fname, bbox_inches="tight", transparent=False)

    ################ CDI ###############
    if do_cdi is True:
        # Create pos and neg helicity set
        pos = (worker_dict["sum_c"] + worker_dict["diff_c"]) / 2
        neg = (worker_dict["sum_c"] - worker_dict["diff_c"]) / 2

        # Create beamstop automatically
        mask_im, mask_topo, mask_pixel, mask_pixel_smooth = create_auto_beamstop(
            pos, neg, mask_draw, use_bs, bs_param
        )

        fig, ax = cimshow(pos*(1-mask_pixel.astype(int)))
        ax.set_title("Verify that this looks like an acceptable beamstop")

        # Get coordinates and create supportmask
        support_coordinates = get_supportmask_coordinates(sample)
        supportmask = create_supportmask(support_coordinates, pos.shape)

        # Do phase retrieval
        (
            retrieved_p,
            retrieved_n,
            retrieved_p_pc,
            retrieved_n_pc,
            bsmask_p,
            bsmask_n,
            gamma_p,
            gamma_n,
        ) = phase_retrieval(
            pos, neg, mask_pixel, supportmask, Startimage=None, Startgamma=None
        )

        # Get Recos partial coherence
        # Positiv partial coherence
        p_pc = fth.reconstructCDI(
            fth.propagate(
                retrieved_p_pc * mask_bs_cdi,
                prop_dist_cdi * 1e-6,
                experimental_setup=experimental_setup,
            )
        )
        # Negative partial coherence
        n_pc = fth.reconstructCDI(
            fth.propagate(
                retrieved_n_pc * mask_bs_cdi,
                prop_dist_cdi * 1e-6,
                experimental_setup=experimental_setup,
            )
        )

        ##### Calc reco and optimze contrast
        recon = helicity[it] * (p_pc - n_pc)
        _, phase_cdi = optimize_phase_contrast(
            recon,
            supportmask,
            method="contrast",
            prefered_color="white",
        )
        # phase_cdi = 0
        recon = recon * np.exp(1j * phase_cdi)
        print("Phase is:", np.round(phase_cdi, 2))
        ########

        # Plot
        plot_recon(recon[roi_cdi], get_title(im_id,topo_id))

        # Save images
        fname = join(
            folder_general,
            "Recon_ImId_%04d_RefId_%s_cdi_stack_%s.png" % (im_id, topo_id, USER),
        )

        print("Saving: %s" % fname)
        plt.savefig(fname, bbox_inches="tight", transparent=False)
        recons_name.append(fname)

        # Save files as h5
        save_cdi_h5()
    else:
        print("Phase retrieval disabled!")

    print(" ")
print("CDI stack processing finished")

In [ ]:
plt.close("all")